In [1]:
import ccxt, abc, time, ssl
from statistics import mean
from threading import Thread, Event

updated = Event()

In [2]:
class Exchange(ccxt.coinsecure):
    def __init__(self):
        self.data = None
        super().__init__()
        
    def check_for_update(self):
        def thread_exec():
            prevTimestamp = None
            while True:
                try:
                    response = self.fetchTicker('BTC/INR')
                    if response['timestamp'] != prevTimestamp:
                        self.data, prevTimestamp = response['info'], response['timestamp']
                        print(self.data)
                        updated.set()
                    else: updated.clear()
                except ssl.SSLEOFError: print('Non-fatal error occured.')
                time.sleep(2)
        thread = Thread(target=thread_exec)
        thread.start()

class Classifier(abc.ABC):
    def __init__(self, exchange): self.exchange = exchange
    
    @abc.abstractmethod
    def predict(self): pass

In [3]:
class _Window(list):
    def __init__(self, window_size=20):
        self.__window_size, self.__actual_window_size = window_size, 0
        super().__init__()
        
    def append(self, price):
        super().append(price)
        if self.__window_size > self.__actual_window_size: self.__actual_window_size += 1
        else: del self[0]
            
class SMA(Classifier):
    def __init__(self, exchange, *, window_size=20):
        self.__window_size = window_size
        self.__sp_window, self.__bp_window = _Window(window_size), _Window(window_size)
        super().__init__(exchange)
    
    def predict(self):
        def classify(avg, price):
            if price < avg: return -1
            if price > avg: return 1
            return 0
        
        def thread_exec():
            sp, bp, predict_sp, predict_bp = None, None, None, None
            while True:
                updated.wait()
                if sp != self.exchange.data['ask']:
                    sp = self.exchange.data['ask']
                    self.__sp_window.append(sp)
                    predict_sp = classify(mean(self.__sp_window), sp)
                if bp != self.exchange.data['bid']:
                    bp = self.exchange.data['bid']
                    self.__bp_window.append(bp)
                    predcit_bp = classify(mean(self.__bp_window), bp)
                print(self.__sp_window, self.__bp_window)
                print('sp: {}'.format(predict_sp), 'bp: {}'.format(predcit_bp))
                time.sleep(2)
        thread = Thread(target=thread_exec)
        thread.start()

In [4]:
exchange = Exchange()

In [5]:
exchange.checkForUpdate()

In [6]:
predictor = SMA(exchange, window_size=3)

In [7]:
predictor.predict()

{'lastPrice': 57650000, 'timestamp': 1518473183640, 'bid': 57610100, 'ask': 58700000, 'fiatvolume': 481711009, 'coinvolume': 833000000, 'open': 58299897, 'high': 59000000, 'low': 55255000}[58700000]
 [57610100]
sp: 0 bp: 0
[58700000] [57610100]
sp: 0 bp: 0
{'lastPrice': 57610085, 'timestamp': 1518474457751, 'bid': 57610100, 'ask': 58700000, 'fiatvolume': 483727362, 'coinvolume': 836500000, 'open': 58299897, 'high': 59000000, 'low': 55255000}
[58700000] [57610100]
sp: 0 bp: 0
[58700000] [57610100]
sp: 0 bp: 0
